In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

# add path for demo utils functions 
import sys
import os
import glob
import json
import numpy as np
import cv2
import random
import math

import matplotlib.pyplot as plt
from datetime import datetime
from scipy.spatial.transform import Rotation
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
inpath = r"D:\OneDrive - University of Utah\HJ\PhD\Lectures\2020Fall\CS6630_DataVis\Homeworks\my_work\hw5\data\polls.json"
with open(inpath, 'r') as f:
    j = json.load(f)
    print(len(j))
    print(len(j[0]))
    print(j[0][0])
    print(len(j[0][1]))
    for idx, v in enumerate(j[0][1]):
        print("{} ===============================".format(idx))
        for k, vv in v.items():
            print("\t{} | {}".format(k, v))
        

50
2

765
0 ===============================
	state | {'state': '', 'name': 'YouGov', 'margin': -8}
	name | {'state': '', 'name': 'YouGov', 'margin': -8}
	margin | {'state': '', 'name': 'YouGov', 'margin': -8}
1 ===============================
	state | {'state': '', 'name': 'Ipsos', 'margin': -7}
	name | {'state': '', 'name': 'Ipsos', 'margin': -7}
	margin | {'state': '', 'name': 'Ipsos', 'margin': -7}
2 ===============================
	state | {'state': '', 'name': 'Rasmussen Reports/Pulse Opinion Research', 'margin': -8}
	name | {'state': '', 'name': 'Rasmussen Reports/Pulse Opinion Research', 'margin': -8}
	margin | {'state': '', 'name': 'Rasmussen Reports/Pulse Opinion Research', 'margin': -8}
3 ===============================
	state | {'state': '', 'name': 'USC Dornsife/Los Angeles Times', 'margin': -7.790000000000006}
	name | {'state': '', 'name': 'USC Dornsife/Los Angeles Times', 'margin': -7.790000000000006}
	margin | {'state': '', 'name': 'USC Dornsife/Los Angeles Times', 'marg

In [34]:
inpath = r"C:\Users\hjoon\Desktop\test.txt"
with open(inpath, 'r') as f:
    ls = f.readlines()
    headers = []
    out_json = []
    for i in range(len(ls)):
        l = ls[i]
        if i == 0:
            headers = l.split()
            print(headers)
        else:
            v = l.split()
            if len(v) == len(headers):
                jdic = {}
                for k in range(len(v)):
                    jdic[headers[k]] = v[i]
                out_json.append(jdic)
                print(jdic)
            else:
                print("NO MATCH:", v)
                print('\t{} {}'.format(len(v), len(headers)))

['Kindred', 'SubjectID', 'Sex', 'VitalStatus', 'DeathMonth', 'DeathYear', 'BMI', 'FirstClinicVisit', 'Age', 'at', 'Clinic', 'visit', 's1_LabID_CRCid', 'Oncoarray_date', 'Smoke', 'Alcohol', 'NSAID', 'HRT', 'Exercise', 'NOTE', 'RR', 'BMI', 'RR', 'SMOKE', 'RR', 'ALCOHOL', 'RR', 'NSAID', 'RR', 'HRT', 'RR', 'EXERCISE', 'Product', 'of', 'RR']
NO MATCH: ['4562', '10002234', 'F', 'Alive', '2/8/1999', '71', '32710', 'Null', 'Null', 'Null', 'Null', 'Null', 'NULL', 'Null', 'Null', 'Null', 'Null', 'Null', '0']
	19 35
NO MATCH: ['4562', '10002235', 'F', 'Alive', '2/9/1999', '71', '32708', 'Null', 'Null', 'Null', 'Null', 'Null', 'NULL', 'Null', 'Null', 'Null', 'Null', 'Null', '0']
	19 35
NO MATCH: ['4562', '10002302', 'F', 'Alive', '5/7/1999', '62', '33652', 'Null', 'Null', 'Null', 'Null', 'Null', 'NULL', 'Null', 'Null', 'Null', 'Null', 'Null', '0']
	19 35
NO MATCH: ['4562', '10002318', 'F', 'Alive', '6/11/1999', '74', '33945', 'Null', 'Null', 'Null', 'Null', 'Null', 'NULL', 'Null', 'Null', 'Null', 

In [45]:
data = []
data.append({"Kindred": "4562", "SubjectID": "10002234", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36199", "Age at Clinic visit": "71", "s1_LabID_CRCid": "32710", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002235", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36200", "Age at Clinic visit": "71", "s1_LabID_CRCid": "32708", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002302", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36287", "Age at Clinic visit": "62", "s1_LabID_CRCid": "33652", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002318", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36322", "Age at Clinic visit": "74", "s1_LabID_CRCid": "33945", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002273", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "5", "DeathYear": "1999", "BMI": None, "FirstClinicVisit": "36233", "Age at Clinic visit": "97", "s1_LabID_CRCid": "33106", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002237", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "4", "DeathYear": "2001", "BMI": None, "FirstClinicVisit": "36233", "Age at Clinic visit": "78", "s1_LabID_CRCid": "33107", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002233", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "6", "DeathYear": "2006", "BMI": None, "FirstClinicVisit": "36208", "Age at Clinic visit": "74", "s1_LabID_CRCid": "32813", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002242", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "1", "DeathYear": "2010", "BMI": None, "FirstClinicVisit": "36234", "Age at Clinic visit": "57", "s1_LabID_CRCid": "33104", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002291", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "8", "DeathYear": "2010", "BMI": None, "FirstClinicVisit": "36263", "Age at Clinic visit": "59", "s1_LabID_CRCid": "33393", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002236", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36207", "Age at Clinic visit": "70", "s1_LabID_CRCid": "32782", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002203", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36325", "Age at Clinic visit": "45", "s1_LabID_CRCid": "33971", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002204", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36229", "Age at Clinic visit": "49", "s1_LabID_CRCid": "33872", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4562", "SubjectID": "10002253", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.97", "FirstClinicVisit": "36229", "Age at Clinic visit": "57", "s1_LabID_CRCid": "33086", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": "11", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.752"})
data.append({"Kindred": "4562", "SubjectID": "10001889", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "35892", "Age at Clinic visit": "67", "s1_LabID_CRCid": "29545", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "23", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.77"})
data.append({"Kindred": "4562", "SubjectID": "10002225", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.36", "FirstClinicVisit": "36222", "Age at Clinic visit": "51", "s1_LabID_CRCid": "33002", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "42", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.8778"})
data.append({"Kindred": "4562", "SubjectID": "10002252", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "11", "DeathYear": "2014", "BMI": "24.33", "FirstClinicVisit": "36243", "Age at Clinic visit": "74", "s1_LabID_CRCid": "33225", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "13", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.79", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.9006"})
data.append({"Kindred": "4562", "SubjectID": "10001886", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2003", "BMI": "24.8", "FirstClinicVisit": "36199", "Age at Clinic visit": "73", "s1_LabID_CRCid": "32709", "Oncoarray_date": "43703", "Smoke": "0.25", "Alcohol": "No", "NSAID": "5", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.92029008"})
data.append({"Kindred": "4562", "SubjectID": "10001893", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "11", "DeathYear": "2014", "BMI": "25.49", "FirstClinicVisit": "36130", "Age at Clinic visit": "68", "s1_LabID_CRCid": "32046", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "15", "HRT": None, "Exercise": "No", "NOTE": "also snp", "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.79", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.9401"})
data.append({"Kindred": "4562", "SubjectID": "10002300", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.46", "FirstClinicVisit": "36291", "Age at Clinic visit": "41", "s1_LabID_CRCid": "33663", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "3", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.952"})
data.append({"Kindred": "4562", "SubjectID": "10002201", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.43", "FirstClinicVisit": "36243", "Age at Clinic visit": "51", "s1_LabID_CRCid": "33227", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "5", "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.992"})
data.append({"Kindred": "4562", "SubjectID": "10002224", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.3", "FirstClinicVisit": "36222", "Age at Clinic visit": "49", "s1_LabID_CRCid": "33003", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "0", "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4562", "SubjectID": "10002226", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.5", "FirstClinicVisit": "36222", "Age at Clinic visit": "44", "s1_LabID_CRCid": "33001", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4562", "SubjectID": "10002278", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.98", "FirstClinicVisit": "36264", "Age at Clinic visit": "37", "s1_LabID_CRCid": "33392", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "10", "NSAID": "No", "HRT": None, "Exercise": "5", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "1.008368"})
data.append({"Kindred": "4562", "SubjectID": "10002264", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.46", "FirstClinicVisit": "36250", "Age at Clinic visit": "56", "s1_LabID_CRCid": "33286", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "6", "NSAID": "No", "HRT": "9", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.01864"})
data.append({"Kindred": "4562", "SubjectID": "10002301", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.14", "FirstClinicVisit": "36292", "Age at Clinic visit": "40", "s1_LabID_CRCid": "33665", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "28", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.07"})
data.append({"Kindred": "4562", "SubjectID": "10002159", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "5", "DeathYear": "2012", "BMI": "32.73", "FirstClinicVisit": "36144", "Age at Clinic visit": "80", "s1_LabID_CRCid": None, "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "60", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "4562", "SubjectID": "10002232", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27", "FirstClinicVisit": "36264", "Age at Clinic visit": "43", "s1_LabID_CRCid": "33391", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "2", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1186"})
data.append({"Kindred": "4562", "SubjectID": "10002239", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "32.43", "FirstClinicVisit": "36207", "Age at Clinic visit": "47", "s1_LabID_CRCid": "32787", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "1", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.128"})
data.append({"Kindred": "4562", "SubjectID": "10001885", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "41.62", "FirstClinicVisit": "36102", "Age at Clinic visit": "62", "s1_LabID_CRCid": "31734", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "11", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.128"})
data.append({"Kindred": "4562", "SubjectID": "75987", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.8", "FirstClinicVisit": "40602", "Age at Clinic visit": "45", "s1_LabID_CRCid": "93865", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "4562", "SubjectID": "10002230", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.13", "FirstClinicVisit": "36207", "Age at Clinic visit": "40", "s1_LabID_CRCid": "32785", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002296", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.29", "FirstClinicVisit": "36306", "Age at Clinic visit": "44", "s1_LabID_CRCid": "33822", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002202", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.18", "FirstClinicVisit": "36242", "Age at Clinic visit": "52", "s1_LabID_CRCid": "33228", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002293", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.32", "FirstClinicVisit": "36284", "Age at Clinic visit": "45", "s1_LabID_CRCid": "33626", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002263", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.65", "FirstClinicVisit": "36284", "Age at Clinic visit": "54", "s1_LabID_CRCid": "33631", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002295", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.49", "FirstClinicVisit": "36278", "Age at Clinic visit": "47", "s1_LabID_CRCid": "33507", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002196", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.54", "FirstClinicVisit": "36242", "Age at Clinic visit": "49", "s1_LabID_CRCid": "33226", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002298", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.36", "FirstClinicVisit": "36278", "Age at Clinic visit": "52", "s1_LabID_CRCid": "33506", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002199", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.5", "FirstClinicVisit": "36214", "Age at Clinic visit": "47", "s1_LabID_CRCid": "32948", "Oncoarray_date": None, "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4562", "SubjectID": "10002136", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2014", "BMI": "36.96", "FirstClinicVisit": "36271", "Age at Clinic visit": "72", "s1_LabID_CRCid": "33666", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "7", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1985"})
data.append({"Kindred": "4562", "SubjectID": "10002206", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.79", "FirstClinicVisit": "36222", "Age at Clinic visit": "53", "s1_LabID_CRCid": "32949", "Oncoarray_date": "PENDING", "Smoke": "1.5", "Alcohol": "8", "NSAID": "No", "HRT": "24", "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.1994272"})
data.append({"Kindred": "4562", "SubjectID": "10002229", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.55", "FirstClinicVisit": "36207", "Age at Clinic visit": "38", "s1_LabID_CRCid": "32784", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "6", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2198"})
data.append({"Kindred": "4562", "SubjectID": "10002189", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "55.24", "FirstClinicVisit": "36250", "Age at Clinic visit": "45", "s1_LabID_CRCid": "33287", "Oncoarray_date": "43703", "Smoke": "0.3", "Alcohol": "No", "NSAID": "17", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.226841"})
data.append({"Kindred": "4562", "SubjectID": "10002294", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.98", "FirstClinicVisit": "36306", "Age at Clinic visit": "50", "s1_LabID_CRCid": "33823", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "4562", "SubjectID": "10002244", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "9", "DeathYear": "2009", "BMI": "38.13", "FirstClinicVisit": "36257", "Age at Clinic visit": "66", "s1_LabID_CRCid": "33323", "Oncoarray_date": None, "Smoke": "5", "Alcohol": "No", "NSAID": "No", "HRT": "24", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.27464"})
data.append({"Kindred": "4562", "SubjectID": "10002251", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.08", "FirstClinicVisit": "36250", "Age at Clinic visit": "71", "s1_LabID_CRCid": "33285", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "1", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "4562", "SubjectID": "10002157", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2013", "BMI": "33", "FirstClinicVisit": "36298", "Age at Clinic visit": "77", "s1_LabID_CRCid": "32047", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "2", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "4562", "SubjectID": "10002257", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.47", "FirstClinicVisit": "36230", "Age at Clinic visit": "59", "s1_LabID_CRCid": "33088", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "3", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "4562", "SubjectID": "10002238", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "2", "DeathYear": "2013", "BMI": "36.51", "FirstClinicVisit": "36234", "Age at Clinic visit": "59", "s1_LabID_CRCid": "33105", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "3", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "4562", "SubjectID": "10002231", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.96", "FirstClinicVisit": "36208", "Age at Clinic visit": "42", "s1_LabID_CRCid": "33322", "Oncoarray_date": None, "Smoke": "1.5", "Alcohol": "4", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.378374"})
data.append({"Kindred": "4562", "SubjectID": "10002195", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.69", "FirstClinicVisit": "36179", "Age at Clinic visit": "46", "s1_LabID_CRCid": "32440", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002163", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.78", "FirstClinicVisit": "36172", "Age at Clinic visit": "42", "s1_LabID_CRCid": "32378", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002130", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "34.3", "FirstClinicVisit": "36102", "Age at Clinic visit": "37", "s1_LabID_CRCid": "31735", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002129", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "34.57", "FirstClinicVisit": "36102", "Age at Clinic visit": "44", "s1_LabID_CRCid": "31733", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002186", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "36.49", "FirstClinicVisit": "36187", "Age at Clinic visit": "40", "s1_LabID_CRCid": "32553", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002205", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "39.74", "FirstClinicVisit": "36222", "Age at Clinic visit": "44", "s1_LabID_CRCid": "32950", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002283", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "45.18", "FirstClinicVisit": "36264", "Age at Clinic visit": "35", "s1_LabID_CRCid": "33394", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002134", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "1", "DeathYear": "2014", "BMI": "31.01", "FirstClinicVisit": "36130", "Age at Clinic visit": "74", "s1_LabID_CRCid": "32048", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002162", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.26", "FirstClinicVisit": "36172", "Age at Clinic visit": "43", "s1_LabID_CRCid": "32379", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002240", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.28", "FirstClinicVisit": "36207", "Age at Clinic visit": "56", "s1_LabID_CRCid": "32783", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002299", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.87", "FirstClinicVisit": "36284", "Age at Clinic visit": "49", "s1_LabID_CRCid": "33627", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002164", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "37.02", "FirstClinicVisit": "36172", "Age at Clinic visit": "40", "s1_LabID_CRCid": "32377", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002185", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "37.62", "FirstClinicVisit": "36186", "Age at Clinic visit": "47", "s1_LabID_CRCid": "32552", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002275", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.3", "FirstClinicVisit": "36340", "Age at Clinic visit": "68", "s1_LabID_CRCid": "34171", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "4562", "SubjectID": "10002254", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.12", "FirstClinicVisit": "36230", "Age at Clinic visit": "44", "s1_LabID_CRCid": "33087", "Oncoarray_date": "43703", "Smoke": "20.8", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "0.5", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.19", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.4756"})
data.append({"Kindred": "4562", "SubjectID": "10002187", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26", "FirstClinicVisit": "36179", "Age at Clinic visit": "43", "s1_LabID_CRCid": "32441", "Oncoarray_date": "43703", "Smoke": "20", "Alcohol": "16", "NSAID": "No", "HRT": None, "Exercise": "1.25", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.19", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.515227"})
data.append({"Kindred": "4562", "SubjectID": "10002197", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.63", "FirstClinicVisit": "36179", "Age at Clinic visit": "47", "s1_LabID_CRCid": "32443", "Oncoarray_date": "43703", "Smoke": "33", "Alcohol": "14", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.19", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.578892"})
data.append({"Kindred": "4562", "SubjectID": "10002265", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.53", "FirstClinicVisit": "36292", "Age at Clinic visit": "50", "s1_LabID_CRCid": "33918", "Oncoarray_date": "WGS", "Smoke": "0.5", "Alcohol": "24", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.704831"})
data.append({"Kindred": "4562", "SubjectID": "10002281", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "10", "DeathYear": "2004", "BMI": "41.19", "FirstClinicVisit": "36292", "Age at Clinic visit": "46", "s1_LabID_CRCid": "33664", "Oncoarray_date": None, "Smoke": "54", "Alcohol": "78", "NSAID": "6", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.38", "RR ALCOHOL": "1.23", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "2.0343339"})
data.append({"Kindred": "4586", "SubjectID": "10002127", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "66", "s1_LabID_CRCid": "31921", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002135", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36101", "Age at Clinic visit": "65", "s1_LabID_CRCid": "31731", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002139", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36116", "Age at Clinic visit": "60", "s1_LabID_CRCid": "31872", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002158", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "72", "s1_LabID_CRCid": None, "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10001941", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "10", "DeathYear": "1999", "BMI": None, "FirstClinicVisit": "36047", "Age at Clinic visit": "82", "s1_LabID_CRCid": "30988", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002160", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "5", "DeathYear": "2005", "BMI": None, "FirstClinicVisit": "36251", "Age at Clinic visit": "81", "s1_LabID_CRCid": "32752", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002137", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "2", "DeathYear": "2010", "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "67", "s1_LabID_CRCid": "32551", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002075", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36109", "Age at Clinic visit": "35", "s1_LabID_CRCid": "31986", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002153", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36123", "Age at Clinic visit": "57", "s1_LabID_CRCid": "31938", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002140", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "9", "DeathYear": "2000", "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "80", "s1_LabID_CRCid": "31920", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002258", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "6", "DeathYear": "2001", "BMI": None, "FirstClinicVisit": "36215", "Age at Clinic visit": "94", "s1_LabID_CRCid": "32956", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10002051", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "50", "s1_LabID_CRCid": "32550", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10001950", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2004", "BMI": None, "FirstClinicVisit": "36111", "Age at Clinic visit": "72", "s1_LabID_CRCid": "31884", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10001939", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2006", "BMI": None, "FirstClinicVisit": "36120", "Age at Clinic visit": "64", "s1_LabID_CRCid": "31939", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4586", "SubjectID": "10001937", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.12", "FirstClinicVisit": "36060", "Age at Clinic visit": "56", "s1_LabID_CRCid": "31162", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "10", "HRT": "1", "Exercise": "3.75", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "0.8", "RR EXERCISE": "0.76", "Product of RR": "0.5168"})
data.append({"Kindred": "4586", "SubjectID": "10001955", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.58", "FirstClinicVisit": "35969", "Age at Clinic visit": "45", "s1_LabID_CRCid": "30316", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "29", "HRT": "1", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.70224"})
data.append({"Kindred": "4586", "SubjectID": "10001964", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.2", "FirstClinicVisit": "36032", "Age at Clinic visit": "56", "s1_LabID_CRCid": "30925", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "40", "HRT": "2", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.73304"})
data.append({"Kindred": "4586", "SubjectID": "10001963", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "19.94", "FirstClinicVisit": "36011", "Age at Clinic visit": "38", "s1_LabID_CRCid": "30715", "Oncoarray_date": None, "Smoke": "25", "Alcohol": "52", "NSAID": "20", "HRT": None, "Exercise": "5", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.19", "RR ALCOHOL": "1.07", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.74513516"})
data.append({"Kindred": "4586", "SubjectID": "10001910", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.14", "FirstClinicVisit": "35906", "Age at Clinic visit": "43", "s1_LabID_CRCid": "29647", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "4586", "SubjectID": "10001960", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.3", "FirstClinicVisit": "35997", "Age at Clinic visit": "43", "s1_LabID_CRCid": "30582", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2.5", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "4586", "SubjectID": "10002073", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "18.11", "FirstClinicVisit": "36088", "Age at Clinic visit": "36", "s1_LabID_CRCid": "31473", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "3.75", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "4586", "SubjectID": "10002094", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.48", "FirstClinicVisit": "36109", "Age at Clinic visit": "33", "s1_LabID_CRCid": "31805", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "5", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "4586", "SubjectID": "10001871", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "10", "DeathYear": "2004", "BMI": "22.28", "FirstClinicVisit": "35941", "Age at Clinic visit": "66", "s1_LabID_CRCid": "30011", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "35", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.77"})
data.append({"Kindred": "4586", "SubjectID": "10002074", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.65", "FirstClinicVisit": "36088", "Age at Clinic visit": "39", "s1_LabID_CRCid": "31474", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "10", "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.8"})
data.append({"Kindred": "4586", "SubjectID": "10002093", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.13", "FirstClinicVisit": "36109", "Age at Clinic visit": "30", "s1_LabID_CRCid": "31804", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8664"})
data.append({"Kindred": "4586", "SubjectID": "10002052", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.75", "FirstClinicVisit": "36046", "Age at Clinic visit": "40", "s1_LabID_CRCid": "31044", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8664"})
data.append({"Kindred": "4586", "SubjectID": "10001954", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.87", "FirstClinicVisit": "35969", "Age at Clinic visit": "37", "s1_LabID_CRCid": "30317", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "5", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9044"})
data.append({"Kindred": "4586", "SubjectID": "10001921", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.32", "FirstClinicVisit": "35948", "Age at Clinic visit": "50", "s1_LabID_CRCid": "30062", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "1", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.912"})
data.append({"Kindred": "4586", "SubjectID": "10001935", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.06", "FirstClinicVisit": "36060", "Age at Clinic visit": "58", "s1_LabID_CRCid": "31163", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "28", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.912"})
data.append({"Kindred": "4586", "SubjectID": "10001938", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.29", "FirstClinicVisit": "35962", "Age at Clinic visit": "67", "s1_LabID_CRCid": "30211", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9424"})
data.append({"Kindred": "4586", "SubjectID": "10001916", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": None, "DeathYear": None, "BMI": "29.32", "FirstClinicVisit": "35941", "Age at Clinic visit": "72", "s1_LabID_CRCid": "30008", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9424"})
data.append({"Kindred": "4586", "SubjectID": "10002070", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.45", "FirstClinicVisit": "36088", "Age at Clinic visit": "43", "s1_LabID_CRCid": "31475", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4586", "SubjectID": "10001971", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "19.74", "FirstClinicVisit": "36046", "Age at Clinic visit": "43", "s1_LabID_CRCid": "31042", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4586", "SubjectID": "10002072", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.4", "FirstClinicVisit": "36088", "Age at Clinic visit": "37", "s1_LabID_CRCid": "31472", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4586", "SubjectID": "10001977", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.67", "FirstClinicVisit": "36025", "Age at Clinic visit": "39", "s1_LabID_CRCid": "30824", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "4586", "SubjectID": "10001912", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "39.05", "FirstClinicVisit": "35948", "Age at Clinic visit": "61", "s1_LabID_CRCid": "30063", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": "5", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.06032"})
data.append({"Kindred": "4586", "SubjectID": "10001959", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.5", "FirstClinicVisit": "36137", "Age at Clinic visit": "49", "s1_LabID_CRCid": "32128", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0716"})
data.append({"Kindred": "4586", "SubjectID": "10002060", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "32.21", "FirstClinicVisit": "36081", "Age at Clinic visit": "47", "s1_LabID_CRCid": "31395", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "27", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "4586", "SubjectID": "10001957", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "36.07", "FirstClinicVisit": "35997", "Age at Clinic visit": "48", "s1_LabID_CRCid": "30581", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "40", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "4586", "SubjectID": "10002156", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.01", "FirstClinicVisit": "36172", "Age at Clinic visit": "33", "s1_LabID_CRCid": "32380", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "20", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "4586", "SubjectID": "10001956", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.67", "FirstClinicVisit": "35969", "Age at Clinic visit": "42", "s1_LabID_CRCid": "30315", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "14", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.128"})
data.append({"Kindred": "4586", "SubjectID": "10002167", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.01", "FirstClinicVisit": "36144", "Age at Clinic visit": "34", "s1_LabID_CRCid": "32189", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "0.5", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "4586", "SubjectID": "10001962", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.64", "FirstClinicVisit": "36011", "Age at Clinic visit": "57", "s1_LabID_CRCid": "30717", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "0.5", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "4586", "SubjectID": "10002061", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.87", "FirstClinicVisit": "36144", "Age at Clinic visit": "39", "s1_LabID_CRCid": "32190", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "4586", "SubjectID": "10001911", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": None, "DeathYear": None, "BMI": "23.91", "FirstClinicVisit": "35941", "Age at Clinic visit": "75", "s1_LabID_CRCid": "30010", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "4586", "SubjectID": "10001966", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.82", "FirstClinicVisit": "36032", "Age at Clinic visit": "60", "s1_LabID_CRCid": "30927", "Oncoarray_date": None, "Smoke": "10", "Alcohol": "14", "NSAID": "48", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.15444868"})
data.append({"Kindred": "4586", "SubjectID": "10002100", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.06", "FirstClinicVisit": "36116", "Age at Clinic visit": "52", "s1_LabID_CRCid": "31874", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "1", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1656"})
data.append({"Kindred": "4586", "SubjectID": "10002097", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.91", "FirstClinicVisit": "36109", "Age at Clinic visit": "34", "s1_LabID_CRCid": "31803", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001913", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.2", "FirstClinicVisit": "35920", "Age at Clinic visit": "45", "s1_LabID_CRCid": "29810", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001967", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.24", "FirstClinicVisit": "36053", "Age at Clinic visit": "49", "s1_LabID_CRCid": "31110", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001920", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.28", "FirstClinicVisit": "35962", "Age at Clinic visit": "65", "s1_LabID_CRCid": "30210", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001914", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.24", "FirstClinicVisit": "35941", "Age at Clinic visit": "41", "s1_LabID_CRCid": "30009", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001934", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2014", "BMI": "26.91", "FirstClinicVisit": "35997", "Age at Clinic visit": "69", "s1_LabID_CRCid": "30583", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10001942", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.5", "FirstClinicVisit": "36089", "Age at Clinic visit": "46", "s1_LabID_CRCid": "31469", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4586", "SubjectID": "10002096", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "36.31", "FirstClinicVisit": "36116", "Age at Clinic visit": "48", "s1_LabID_CRCid": "31871", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "8", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1985"})
data.append({"Kindred": "4586", "SubjectID": "10001936", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "32.28", "FirstClinicVisit": "35955", "Age at Clinic visit": "62", "s1_LabID_CRCid": "30108", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "6", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1985"})
data.append({"Kindred": "4586", "SubjectID": "10001999", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.29", "FirstClinicVisit": "36011", "Age at Clinic visit": "36", "s1_LabID_CRCid": "30716", "Oncoarray_date": None, "Smoke": "18.75", "Alcohol": "12", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2091"})
data.append({"Kindred": "4586", "SubjectID": "10001997", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.19", "FirstClinicVisit": "36018", "Age at Clinic visit": "41", "s1_LabID_CRCid": "30764", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "4586", "SubjectID": "10002133", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.5", "FirstClinicVisit": "36130", "Age at Clinic visit": "30", "s1_LabID_CRCid": "32045", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "4586", "SubjectID": "10001958", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28", "FirstClinicVisit": "36053", "Age at Clinic visit": "53", "s1_LabID_CRCid": "31111", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "4586", "SubjectID": "10001926", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "1", "DeathYear": "2007", "BMI": "28.57", "FirstClinicVisit": "36032", "Age at Clinic visit": "80", "s1_LabID_CRCid": "30924", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "4586", "SubjectID": "10001969", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.05", "FirstClinicVisit": "36025", "Age at Clinic visit": "52", "s1_LabID_CRCid": "30823", "Oncoarray_date": "43458", "Smoke": "20.25", "Alcohol": "No", "NSAID": "36", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.19", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.291983"})
data.append({"Kindred": "4586", "SubjectID": "10001917", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "9", "DeathYear": "2007", "BMI": "22.2", "FirstClinicVisit": "36137", "Age at Clinic visit": "62", "s1_LabID_CRCid": "32130", "Oncoarray_date": "43703", "Smoke": "45", "Alcohol": "112", "NSAID": "41", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.38", "RR ALCOHOL": "1.23", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.306998"})
data.append({"Kindred": "4586", "SubjectID": "10001991", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "40.72", "FirstClinicVisit": "36046", "Age at Clinic visit": "49", "s1_LabID_CRCid": "31043", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "4586", "SubjectID": "10002053", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "36.04", "FirstClinicVisit": "36053", "Age at Clinic visit": "54", "s1_LabID_CRCid": "31109", "Oncoarray_date": "43458", "Smoke": "9", "Alcohol": "22", "NSAID": "No", "HRT": "2", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.3638648"})
data.append({"Kindred": "4586", "SubjectID": "10001968", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.16", "FirstClinicVisit": "36025", "Age at Clinic visit": "54", "s1_LabID_CRCid": "30825", "Oncoarray_date": "43458", "Smoke": "7.5", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "0.75", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.4012"})
data.append({"Kindred": "4586", "SubjectID": "10001965", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.65", "FirstClinicVisit": "36060", "Age at Clinic visit": "38", "s1_LabID_CRCid": "31164", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10002058", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.81", "FirstClinicVisit": "36081", "Age at Clinic visit": "53", "s1_LabID_CRCid": "31394", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10002059", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "37.81", "FirstClinicVisit": "36081", "Age at Clinic visit": "55", "s1_LabID_CRCid": "31396", "Oncoarray_date": "43458", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10002155", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.26", "FirstClinicVisit": "36172", "Age at Clinic visit": "31", "s1_LabID_CRCid": "92966", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10002099", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.86", "FirstClinicVisit": "36116", "Age at Clinic visit": "38", "s1_LabID_CRCid": "31873", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10001925", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.69", "FirstClinicVisit": "35948", "Age at Clinic visit": "48", "s1_LabID_CRCid": "30061", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "4586", "SubjectID": "10001970", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.69", "FirstClinicVisit": "36032", "Age at Clinic visit": "58", "s1_LabID_CRCid": "30926", "Oncoarray_date": "43458", "Smoke": "12", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.5933"})
data.append({"Kindred": "4586", "SubjectID": "10002220", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.63", "FirstClinicVisit": "36214", "Age at Clinic visit": "53", "s1_LabID_CRCid": "32951", "Oncoarray_date": "PENDING", "Smoke": "54", "Alcohol": "34", "NSAID": "3", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.38", "RR ALCOHOL": "1.07", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.72112496"})
data.append({"Kindred": "4853", "SubjectID": "48891", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "40244", "Age at Clinic visit": "36", "s1_LabID_CRCid": "90259", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "10002515", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36581", "Age at Clinic visit": "74", "s1_LabID_CRCid": "41437", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "48892", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "40245", "Age at Clinic visit": "37", "s1_LabID_CRCid": "90260", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "48894", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "40245", "Age at Clinic visit": "31", "s1_LabID_CRCid": "90261", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "10002526", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36627", "Age at Clinic visit": "40", "s1_LabID_CRCid": "38927", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "10002615", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36831", "Age at Clinic visit": "66", "s1_LabID_CRCid": "41438", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "10002616", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2001", "BMI": None, "FirstClinicVisit": "36823", "Age at Clinic visit": "74", "s1_LabID_CRCid": "41387", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "4853", "SubjectID": "10002412", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.46", "FirstClinicVisit": "36502", "Age at Clinic visit": "41", "s1_LabID_CRCid": "36103", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "40", "Exercise": "7", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "0.76", "Product of RR": "0.608"})
data.append({"Kindred": "4853", "SubjectID": "10578", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.6", "FirstClinicVisit": "39371", "Age at Clinic visit": "40", "s1_LabID_CRCid": "79993", "Oncoarray_date": None, "Smoke": None, "Alcohol": "4", "NSAID": "No", "HRT": "No", "Exercise": "5", "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8132"})
data.append({"Kindred": "4853", "SubjectID": "10002087", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.83", "FirstClinicVisit": "36109", "Age at Clinic visit": "48", "s1_LabID_CRCid": "31802", "Oncoarray_date": "43703", "Smoke": "1", "Alcohol": "8", "NSAID": "27", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8420027308"})
data.append({"Kindred": "4853", "SubjectID": "10577", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24", "FirstClinicVisit": "39371", "Age at Clinic visit": "63", "s1_LabID_CRCid": "79995", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": "12", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.85728"})
data.append({"Kindred": "4853", "SubjectID": "10002414", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.13", "FirstClinicVisit": "36502", "Age at Clinic visit": "43", "s1_LabID_CRCid": "36102", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8664"})
data.append({"Kindred": "4853", "SubjectID": "13752", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.55", "FirstClinicVisit": "37260", "Age at Clinic visit": "50", "s1_LabID_CRCid": "46858", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "7", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9044"})
data.append({"Kindred": "4853", "SubjectID": "42255", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.5", "FirstClinicVisit": "39518", "Age at Clinic visit": "34", "s1_LabID_CRCid": "81713", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "2", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.927048"})
data.append({"Kindred": "4853", "SubjectID": "10001891", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.59", "FirstClinicVisit": "36137", "Age at Clinic visit": "67", "s1_LabID_CRCid": "31144", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "4853", "SubjectID": "13751", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.39", "FirstClinicVisit": "37369", "Age at Clinic visit": "56", "s1_LabID_CRCid": "48378", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "4853", "SubjectID": "10002085", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "19.92", "FirstClinicVisit": "36102", "Age at Clinic visit": "45", "s1_LabID_CRCid": "31732", "Oncoarray_date": "43703", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "4853", "SubjectID": "10236", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "19.7", "FirstClinicVisit": "39573", "Age at Clinic visit": "68", "s1_LabID_CRCid": "82432", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "4853", "SubjectID": "10580", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29", "FirstClinicVisit": "39371", "Age at Clinic visit": "34", "s1_LabID_CRCid": "79992", "Oncoarray_date": None, "Smoke": None, "Alcohol": "56", "NSAID": "No", "HRT": None, "Exercise": "3.75", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "1.008368"})
data.append({"Kindred": "4853", "SubjectID": "13754", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.09", "FirstClinicVisit": "37161", "Age at Clinic visit": "38", "s1_LabID_CRCid": "45016", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "4", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.07"})
data.append({"Kindred": "4853", "SubjectID": "10002297", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "39.48", "FirstClinicVisit": "36306", "Age at Clinic visit": "55", "s1_LabID_CRCid": "33824", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "15", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.128"})
data.append({"Kindred": "4853", "SubjectID": "10002516", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.96", "FirstClinicVisit": "36628", "Age at Clinic visit": "45", "s1_LabID_CRCid": "38926", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "4853", "SubjectID": "13753", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.76", "FirstClinicVisit": "37161", "Age at Clinic visit": "46", "s1_LabID_CRCid": "45879", "Oncoarray_date": "43703", "Smoke": "28", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "0.5", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.19", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "10002523", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.3", "FirstClinicVisit": "36769", "Age at Clinic visit": "40", "s1_LabID_CRCid": "40959", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "10002550", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.4", "FirstClinicVisit": "36693", "Age at Clinic visit": "39", "s1_LabID_CRCid": "39637", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "10002522", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.22", "FirstClinicVisit": "36628", "Age at Clinic visit": "53", "s1_LabID_CRCid": "38925", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "10001894", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.41", "FirstClinicVisit": "36137", "Age at Clinic visit": "63", "s1_LabID_CRCid": "32127", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "11889", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.4", "FirstClinicVisit": "39532", "Age at Clinic visit": "38", "s1_LabID_CRCid": "81908", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "4853", "SubjectID": "10002086", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.35", "FirstClinicVisit": "36137", "Age at Clinic visit": "60", "s1_LabID_CRCid": "32129", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "4853", "SubjectID": "13750", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.24", "FirstClinicVisit": "37369", "Age at Clinic visit": "60", "s1_LabID_CRCid": "45017", "Oncoarray_date": "43703", "Smoke": "30", "Alcohol": "14", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.19", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2733"})
data.append({"Kindred": "4853", "SubjectID": "10002088", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.58", "FirstClinicVisit": "36509", "Age at Clinic visit": "56", "s1_LabID_CRCid": "36330", "Oncoarray_date": "WGS", "Smoke": "0.8", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3447"})
data.append({"Kindred": "4853", "SubjectID": "10002207", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.68", "FirstClinicVisit": "36179", "Age at Clinic visit": "41", "s1_LabID_CRCid": "32442", "Oncoarray_date": "43703", "Smoke": "4", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3447"})
data.append({"Kindred": "4853", "SubjectID": "10002413", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.63", "FirstClinicVisit": "36502", "Age at Clinic visit": "45", "s1_LabID_CRCid": "36101", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "0.75", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4853", "SubjectID": "10579", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.1", "FirstClinicVisit": "39371", "Age at Clinic visit": "38", "s1_LabID_CRCid": "79994", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4853", "SubjectID": "10002290", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.69", "FirstClinicVisit": "36284", "Age at Clinic visit": "46", "s1_LabID_CRCid": "33628", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "4853", "SubjectID": "10002566", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "34.97", "FirstClinicVisit": "36748", "Age at Clinic visit": "58", "s1_LabID_CRCid": "40596", "Oncoarray_date": "43703", "Smoke": "No", "Alcohol": "50", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.5087"})
data.append({"Kindred": "5234", "SubjectID": "10002192", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2003", "BMI": None, "FirstClinicVisit": "36794", "Age at Clinic visit": "42", "s1_LabID_CRCid": "45708", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "10002626", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "1", "DeathYear": "2009", "BMI": None, "FirstClinicVisit": "39156", "Age at Clinic visit": "59", "s1_LabID_CRCid": "76328", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "10002316", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2014", "BMI": None, "FirstClinicVisit": "39115", "Age at Clinic visit": "85", "s1_LabID_CRCid": "75833", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "11179", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "37119", "Age at Clinic visit": "87", "s1_LabID_CRCid": "45408", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "14837", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "37287", "Age at Clinic visit": "77", "s1_LabID_CRCid": "47596", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "10001897", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36285", "Age at Clinic visit": "71", "s1_LabID_CRCid": "33629", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "13894", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "37111", "Age at Clinic visit": "36", "s1_LabID_CRCid": "45321", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "10003006", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36950", "Age at Clinic visit": "40", "s1_LabID_CRCid": "42510", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "13948", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "8", "DeathYear": "2001", "BMI": None, "FirstClinicVisit": "37120", "Age at Clinic visit": "70", "s1_LabID_CRCid": "45481", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "10002623", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "7", "DeathYear": "2014", "BMI": None, "FirstClinicVisit": "37287", "Age at Clinic visit": "74", "s1_LabID_CRCid": "47595", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5234", "SubjectID": "11185", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.7", "FirstClinicVisit": "37081", "Age at Clinic visit": "56", "s1_LabID_CRCid": "45014", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "36", "HRT": "10", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.616"})
data.append({"Kindred": "5234", "SubjectID": "14553", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.84", "FirstClinicVisit": "37264", "Age at Clinic visit": "41", "s1_LabID_CRCid": "46902", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "18", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.696388"})
data.append({"Kindred": "5234", "SubjectID": "10002553", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36740", "Age at Clinic visit": "63", "s1_LabID_CRCid": "40384", "Oncoarray_date": None, "Smoke": "17.25", "Alcohol": "44", "NSAID": "40", "HRT": "17", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.7448056"})
data.append({"Kindred": "5234", "SubjectID": "10002390", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.66", "FirstClinicVisit": "36460", "Age at Clinic visit": "42", "s1_LabID_CRCid": "35474", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "5234", "SubjectID": "10625", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.61", "FirstClinicVisit": "36924", "Age at Clinic visit": "41", "s1_LabID_CRCid": "42256", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "16", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.77"})
data.append({"Kindred": "5234", "SubjectID": "10002309", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.5", "FirstClinicVisit": "36438", "Age at Clinic visit": "55", "s1_LabID_CRCid": "35494", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "5", "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.8"})
data.append({"Kindred": "5234", "SubjectID": "32612", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "3", "DeathYear": "2014", "BMI": None, "FirstClinicVisit": "38980", "Age at Clinic visit": "68", "s1_LabID_CRCid": "74335", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "7", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.8"})
data.append({"Kindred": "5234", "SubjectID": "10002374", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "3", "DeathYear": "2013", "BMI": "24.44", "FirstClinicVisit": "36446", "Age at Clinic visit": "72", "s1_LabID_CRCid": "35375", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "5", "HRT": "24", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.85728"})
data.append({"Kindred": "5234", "SubjectID": "10002706", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.74", "FirstClinicVisit": "36923", "Age at Clinic visit": "49", "s1_LabID_CRCid": "42253", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "2", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.8664"})
data.append({"Kindred": "5234", "SubjectID": "10002540", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "8", "DeathYear": "2014", "BMI": "30.2", "FirstClinicVisit": "36957", "Age at Clinic visit": "76", "s1_LabID_CRCid": "42582", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "43", "HRT": "20", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.86856"})
data.append({"Kindred": "5234", "SubjectID": "10002363", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.57", "FirstClinicVisit": "36460", "Age at Clinic visit": "42", "s1_LabID_CRCid": "35472", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "5", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9424"})
data.append({"Kindred": "5234", "SubjectID": "10002877", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.87", "FirstClinicVisit": "36860", "Age at Clinic visit": "46", "s1_LabID_CRCid": "41687", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "5", "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.952"})
data.append({"Kindred": "5234", "SubjectID": "10002898", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.79", "FirstClinicVisit": "36860", "Age at Clinic visit": "44", "s1_LabID_CRCid": "41688", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "30", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.9548"})
data.append({"Kindred": "5234", "SubjectID": "35954", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.6", "FirstClinicVisit": "39015", "Age at Clinic visit": "80", "s1_LabID_CRCid": "74698", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "20", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.9548"})
data.append({"Kindred": "5234", "SubjectID": "11186", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.36", "FirstClinicVisit": "36977", "Age at Clinic visit": "45", "s1_LabID_CRCid": "42859", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "9", "HRT": "7", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.9588"})
data.append({"Kindred": "5234", "SubjectID": "10002395", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "47.29", "FirstClinicVisit": "36460", "Age at Clinic visit": "63", "s1_LabID_CRCid": "35476", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "6", "HRT": "9", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.9588"})
data.append({"Kindred": "5234", "SubjectID": "10834", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.26", "FirstClinicVisit": "36924", "Age at Clinic visit": "47", "s1_LabID_CRCid": "42255", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "7", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.969"})
data.append({"Kindred": "5234", "SubjectID": "10002899", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.14", "FirstClinicVisit": "36860", "Age at Clinic visit": "39", "s1_LabID_CRCid": "41686", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "10002552", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2007", "BMI": None, "FirstClinicVisit": "37119", "Age at Clinic visit": "83", "s1_LabID_CRCid": "45409", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1.25", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "32613", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38888", "Age at Clinic visit": "42", "s1_LabID_CRCid": "73274", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "37957", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39125", "Age at Clinic visit": "46", "s1_LabID_CRCid": "75988", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "11074", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.8", "FirstClinicVisit": "36963", "Age at Clinic visit": "58", "s1_LabID_CRCid": "42654", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "11075", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.8", "FirstClinicVisit": "36963", "Age at Clinic visit": "49", "s1_LabID_CRCid": "42653", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "13986", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.3", "FirstClinicVisit": "37152", "Age at Clinic visit": "41", "s1_LabID_CRCid": "45802", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "10002376", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.9", "FirstClinicVisit": "36467", "Age at Clinic visit": "59", "s1_LabID_CRCid": "35574", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "5234", "SubjectID": "11048", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.9", "FirstClinicVisit": "36957", "Age at Clinic visit": "59", "s1_LabID_CRCid": "42581", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "8", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.054"})
data.append({"Kindred": "5234", "SubjectID": "10002548", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.76", "FirstClinicVisit": "36643", "Age at Clinic visit": "54", "s1_LabID_CRCid": "39324", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "10", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.054"})
data.append({"Kindred": "5234", "SubjectID": "10002386", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "34.4", "FirstClinicVisit": "39013", "Age at Clinic visit": "73", "s1_LabID_CRCid": "74685", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "3", "HRT": "3", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.06032"})
data.append({"Kindred": "5234", "SubjectID": "10002389", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.2", "FirstClinicVisit": "36460", "Age at Clinic visit": "45", "s1_LabID_CRCid": "35477", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "30", "HRT": None, "Exercise": "0.75", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "5234", "SubjectID": "10280", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.7", "FirstClinicVisit": "39013", "Age at Clinic visit": "73", "s1_LabID_CRCid": "74687", "Oncoarray_date": None, "Smoke": None, "Alcohol": "No", "NSAID": "20", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.0857"})
data.append({"Kindred": "5234", "SubjectID": "11184", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.2", "FirstClinicVisit": "37012", "Age at Clinic visit": "57", "s1_LabID_CRCid": "44237", "Oncoarray_date": None, "Smoke": "36", "Alcohol": "48", "NSAID": "2", "HRT": "8", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.38", "RR ALCOHOL": "1.07", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.1104032"})
data.append({"Kindred": "5234", "SubjectID": "10002378", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.43", "FirstClinicVisit": "36460", "Age at Clinic visit": "71", "s1_LabID_CRCid": "35478", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "1", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1186"})
data.append({"Kindred": "5234", "SubjectID": "10002559", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.24", "FirstClinicVisit": "36845", "Age at Clinic visit": "69", "s1_LabID_CRCid": "41585", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "2", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1186"})
data.append({"Kindred": "5234", "SubjectID": "10002384", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "36.27", "FirstClinicVisit": "36446", "Age at Clinic visit": "75", "s1_LabID_CRCid": "35373", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "10", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.128"})
data.append({"Kindred": "5234", "SubjectID": "10003007", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.07", "FirstClinicVisit": "37040", "Age at Clinic visit": "36", "s1_LabID_CRCid": "44527", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "10002708", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.09", "FirstClinicVisit": "36924", "Age at Clinic visit": "44", "s1_LabID_CRCid": "42254", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "10001961", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.38", "FirstClinicVisit": "36250", "Age at Clinic visit": "49", "s1_LabID_CRCid": "33288", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "10003000", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.5", "FirstClinicVisit": "37061", "Age at Clinic visit": "37", "s1_LabID_CRCid": "44729", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "14290", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.4", "FirstClinicVisit": "37187", "Age at Clinic visit": "39", "s1_LabID_CRCid": "46362", "Oncoarray_date": None, "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "11513", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23", "FirstClinicVisit": "37019", "Age at Clinic visit": "49", "s1_LabID_CRCid": "44324", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "5234", "SubjectID": "10002578", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.98", "FirstClinicVisit": "36860", "Age at Clinic visit": "68", "s1_LabID_CRCid": "41685", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "3", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1656"})
data.append({"Kindred": "5234", "SubjectID": "14552", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.72", "FirstClinicVisit": "37264", "Age at Clinic visit": "43", "s1_LabID_CRCid": "46903", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "5234", "SubjectID": "10002621", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.6", "FirstClinicVisit": "37166", "Age at Clinic visit": "75", "s1_LabID_CRCid": "46089", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "5234", "SubjectID": "13988", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.16", "FirstClinicVisit": "37173", "Age at Clinic visit": "44", "s1_LabID_CRCid": "46209", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5234", "SubjectID": "11660", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.1", "FirstClinicVisit": "37019", "Age at Clinic visit": "46", "s1_LabID_CRCid": "44323", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5234", "SubjectID": "10002394", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.76", "FirstClinicVisit": "36467", "Age at Clinic visit": "64", "s1_LabID_CRCid": "35575", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5234", "SubjectID": "11183", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.1", "FirstClinicVisit": "37012", "Age at Clinic visit": "48", "s1_LabID_CRCid": "44236", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "28", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2198"})
data.append({"Kindred": "5234", "SubjectID": "11047", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.7", "FirstClinicVisit": "36963", "Age at Clinic visit": "44", "s1_LabID_CRCid": "42655", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "40", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2198"})
data.append({"Kindred": "5234", "SubjectID": "10002622", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.83", "FirstClinicVisit": "36923", "Age at Clinic visit": "50", "s1_LabID_CRCid": "42257", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "10002608", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "10", "DeathYear": "2012", "BMI": "28.1", "FirstClinicVisit": "36845", "Age at Clinic visit": "71", "s1_LabID_CRCid": "41586", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "10002380", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29", "FirstClinicVisit": "36462", "Age at Clinic visit": "83", "s1_LabID_CRCid": "35495", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "10943", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "3", "DeathYear": "2004", "BMI": "28.17", "FirstClinicVisit": "37061", "Age at Clinic visit": "77", "s1_LabID_CRCid": "44732", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "10002362", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.29", "FirstClinicVisit": "36509", "Age at Clinic visit": "50", "s1_LabID_CRCid": "36331", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "14291", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.16", "FirstClinicVisit": "37173", "Age at Clinic visit": "36", "s1_LabID_CRCid": "46211", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "11723", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.4", "FirstClinicVisit": "37061", "Age at Clinic visit": "41", "s1_LabID_CRCid": "44731", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "14073", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.53", "FirstClinicVisit": "37187", "Age at Clinic visit": "36", "s1_LabID_CRCid": "46363", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "14318", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.91", "FirstClinicVisit": "37173", "Age at Clinic visit": "42", "s1_LabID_CRCid": "46210", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5234", "SubjectID": "10002534", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.37", "FirstClinicVisit": "36649", "Age at Clinic visit": "71", "s1_LabID_CRCid": "39102", "Oncoarray_date": None, "Smoke": "31", "Alcohol": "No", "NSAID": "2", "HRT": "33", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.19", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "1.2617808"})
data.append({"Kindred": "5234", "SubjectID": "10002579", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.51", "FirstClinicVisit": "36860", "Age at Clinic visit": "64", "s1_LabID_CRCid": "41684", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "4", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "5234", "SubjectID": "13987", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "32.89", "FirstClinicVisit": "37161", "Age at Clinic visit": "43", "s1_LabID_CRCid": "45880", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "10002377", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2004", "BMI": "41.39", "FirstClinicVisit": "36446", "Age at Clinic visit": "68", "s1_LabID_CRCid": "35374", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "10002322", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "42.43", "FirstClinicVisit": "36467", "Age at Clinic visit": "72", "s1_LabID_CRCid": "35573", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "10002388", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.47", "FirstClinicVisit": "36461", "Age at Clinic visit": "48", "s1_LabID_CRCid": "35475", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "10002381", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.03", "FirstClinicVisit": "36467", "Age at Clinic visit": "68", "s1_LabID_CRCid": "35572", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "10003002", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.47", "FirstClinicVisit": "37061", "Age at Clinic visit": "36", "s1_LabID_CRCid": "44730", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "5234", "SubjectID": "11180", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.96", "FirstClinicVisit": "37040", "Age at Clinic visit": "61", "s1_LabID_CRCid": "44526", "Oncoarray_date": None, "Smoke": "4", "Alcohol": "48", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.704831"})
data.append({"Kindred": "5234", "SubjectID": "10002587", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "32.3", "FirstClinicVisit": "37110", "Age at Clinic visit": "57", "s1_LabID_CRCid": "45322", "Oncoarray_date": "PENDING", "Smoke": "28.5", "Alcohol": "160", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1.19", "RR ALCOHOL": "1.23", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "2.063817"})
data.append({"Kindred": "5275", "SubjectID": "35704", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38877", "Age at Clinic visit": "47", "s1_LabID_CRCid": "73131", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "183", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38994", "Age at Clinic visit": "50", "s1_LabID_CRCid": "74476", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "17632", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "37648", "Age at Clinic visit": "45", "s1_LabID_CRCid": "51571", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "21317", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38876", "Age at Clinic visit": "57", "s1_LabID_CRCid": "73129", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "35546", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38881", "Age at Clinic visit": "40", "s1_LabID_CRCid": "73203", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "35548", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38881", "Age at Clinic visit": "59", "s1_LabID_CRCid": "73199", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "37887", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39106", "Age at Clinic visit": "52", "s1_LabID_CRCid": "75725", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "18114", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2015", "BMI": None, "FirstClinicVisit": "37733", "Age at Clinic visit": "56", "s1_LabID_CRCid": "52348", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "10068", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39112", "Age at Clinic visit": "82", "s1_LabID_CRCid": "75800", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "10216", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "37426", "Age at Clinic visit": "50", "s1_LabID_CRCid": "49122", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "15723", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39960", "Age at Clinic visit": "25", "s1_LabID_CRCid": "86930", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "18576", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38642", "Age at Clinic visit": "80", "s1_LabID_CRCid": "70286", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "18578", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38884", "Age at Clinic visit": "53", "s1_LabID_CRCid": "73236", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "21319", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38887", "Age at Clinic visit": "52", "s1_LabID_CRCid": "73130", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "15722", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38741", "Age at Clinic visit": "25", "s1_LabID_CRCid": "71377", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "35706", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38876", "Age at Clinic visit": "60", "s1_LabID_CRCid": "73128", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "5275", "SubjectID": "35736", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.3", "FirstClinicVisit": "38930", "Age at Clinic visit": "27", "s1_LabID_CRCid": "73869", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.76"})
data.append({"Kindred": "5275", "SubjectID": "10002259", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "36277", "Age at Clinic visit": "44", "s1_LabID_CRCid": "33504", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "43", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.77"})
data.append({"Kindred": "5275", "SubjectID": "15550", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.82", "FirstClinicVisit": "37425", "Age at Clinic visit": "26", "s1_LabID_CRCid": "49121", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "2", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.9044"})
data.append({"Kindred": "5275", "SubjectID": "7956", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21", "FirstClinicVisit": "38979", "Age at Clinic visit": "29", "s1_LabID_CRCid": "74334", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "0.5", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "35738", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "17.7", "FirstClinicVisit": "38965", "Age at Clinic visit": "26", "s1_LabID_CRCid": "74187", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "37236", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.6", "FirstClinicVisit": "39222", "Age at Clinic visit": "34", "s1_LabID_CRCid": "77991", "Oncoarray_date": "WGS", "Smoke": "no", "Alcohol": "no", "NSAID": "no", "HRT": "no", "Exercise": "no", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "7958", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "21.3", "FirstClinicVisit": "39469", "Age at Clinic visit": "26", "s1_LabID_CRCid": "81168", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "7957", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39133", "Age at Clinic visit": "28", "s1_LabID_CRCid": "76076", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "35885", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "6", "DeathYear": "2007", "BMI": None, "FirstClinicVisit": "38981", "Age at Clinic visit": "63", "s1_LabID_CRCid": "74336", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "10001898", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "5", "DeathYear": "2014", "BMI": None, "FirstClinicVisit": "36277", "Age at Clinic visit": "72", "s1_LabID_CRCid": "33505", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": "No", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": "1", "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "15721", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "18.2", "FirstClinicVisit": "39133", "Age at Clinic visit": "29", "s1_LabID_CRCid": "76077", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "10002533", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.27", "FirstClinicVisit": "37685", "Age at Clinic visit": "80", "s1_LabID_CRCid": "51862", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "5275", "SubjectID": "18349", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26", "FirstClinicVisit": "38931", "Age at Clinic visit": "63", "s1_LabID_CRCid": "73200", "Oncoarray_date": "PENDING", "Smoke": "18", "Alcohol": "24", "NSAID": "13", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "0.79", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.13667491"})
data.append({"Kindred": "5275", "SubjectID": "35547", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.2", "FirstClinicVisit": "38881", "Age at Clinic visit": "35", "s1_LabID_CRCid": "73202", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "5275", "SubjectID": "40903", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.8", "FirstClinicVisit": "39413", "Age at Clinic visit": "37", "s1_LabID_CRCid": "80652", "Oncoarray_date": "PENDING", "Smoke": "nO", "Alcohol": "NO", "NSAID": "NO", "HRT": None, "Exercise": "0.25", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "36493", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.4", "FirstClinicVisit": "38965", "Age at Clinic visit": "30", "s1_LabID_CRCid": "74188", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "1", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "18188", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.9", "FirstClinicVisit": "37754", "Age at Clinic visit": "25", "s1_LabID_CRCid": "52621", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "18030", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "10", "DeathYear": "2009", "BMI": "25.1", "FirstClinicVisit": "37733", "Age at Clinic visit": "36", "s1_LabID_CRCid": "52350", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "35737", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.1", "FirstClinicVisit": "39070", "Age at Clinic visit": "34", "s1_LabID_CRCid": "75274", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "21318", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.2", "FirstClinicVisit": "39105", "Age at Clinic visit": "56", "s1_LabID_CRCid": "75717", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": "No", "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": "1", "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "5275", "SubjectID": "35735", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.8", "FirstClinicVisit": "38888", "Age at Clinic visit": "31", "s1_LabID_CRCid": "73275", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.24"})
data.append({"Kindred": "5275", "SubjectID": "10002285", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "4", "DeathYear": "2017", "BMI": "28.47", "FirstClinicVisit": "36284", "Age at Clinic visit": "46", "s1_LabID_CRCid": "33630", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "5275", "SubjectID": "18187", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.41", "FirstClinicVisit": "37754", "Age at Clinic visit": "33", "s1_LabID_CRCid": "52620", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "16", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.2733"})
data.append({"Kindred": "5275", "SubjectID": "35725", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.9", "FirstClinicVisit": "38881", "Age at Clinic visit": "38", "s1_LabID_CRCid": "73201", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "5275", "SubjectID": "40830", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "35.2", "FirstClinicVisit": "39351", "Age at Clinic visit": "35", "s1_LabID_CRCid": "80376", "Oncoarray_date": "PENDING", "Smoke": "NO", "Alcohol": "YES", "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "5275", "SubjectID": "18006", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "27.42", "FirstClinicVisit": "37733", "Age at Clinic visit": "56", "s1_LabID_CRCid": "52349", "Oncoarray_date": "PENDING", "Smoke": "25", "Alcohol": "40", "NSAID": "5", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1.19", "RR ALCOHOL": "1.07", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.42431338"})
data.append({"Kindred": "5275", "SubjectID": "17660", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.78", "FirstClinicVisit": "37664", "Age at Clinic visit": "53", "s1_LabID_CRCid": "51695", "Oncoarray_date": "PENDING", "Smoke": "42", "Alcohol": "28", "NSAID": "No", "HRT": None, "Exercise": "0.75", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1.38", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.683324"})
data.append({"Kindred": "5275", "SubjectID": "41625", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30.8", "FirstClinicVisit": "39434", "Age at Clinic visit": "27", "s1_LabID_CRCid": "80870", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "300", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1.23", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.7343"})
data.append({"Kindred": "7646", "SubjectID": "30514", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39000", "Age at Clinic visit": "67", "s1_LabID_CRCid": "75059", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7646", "SubjectID": "30459", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39175", "Age at Clinic visit": "75", "s1_LabID_CRCid": "76811", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7646", "SubjectID": "30496", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38932", "Age at Clinic visit": "53", "s1_LabID_CRCid": "73686", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7646", "SubjectID": "22835", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "17.8", "FirstClinicVisit": "38874", "Age at Clinic visit": "55", "s1_LabID_CRCid": "73058", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "20", "HRT": "5", "Exercise": "3.75", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "0.76", "Product of RR": "0.46816"})
data.append({"Kindred": "7646", "SubjectID": "29151", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.5", "FirstClinicVisit": "38609", "Age at Clinic visit": "64", "s1_LabID_CRCid": "69873", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "5", "Exercise": "2", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "0.76", "Product of RR": "0.72352"})
data.append({"Kindred": "7646", "SubjectID": "22833", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.6", "FirstClinicVisit": "38874", "Age at Clinic visit": "59", "s1_LabID_CRCid": "73057", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "14", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.8"})
data.append({"Kindred": "7646", "SubjectID": "29152", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.6", "FirstClinicVisit": "38590", "Age at Clinic visit": "68", "s1_LabID_CRCid": "70062", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "5", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.952"})
data.append({"Kindred": "7646", "SubjectID": "29246", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.6", "FirstClinicVisit": "38602", "Age at Clinic visit": "58", "s1_LabID_CRCid": "69959", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "8", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.952"})
data.append({"Kindred": "7646", "SubjectID": "19834", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "8", "DeathYear": "2015", "BMI": None, "FirstClinicVisit": "37880", "Age at Clinic visit": "85", "s1_LabID_CRCid": "57707", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "7646", "SubjectID": "22839", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.7", "FirstClinicVisit": "38464", "Age at Clinic visit": "41", "s1_LabID_CRCid": "68060", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1"})
data.append({"Kindred": "7646", "SubjectID": "22836", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.4", "FirstClinicVisit": "38874", "Age at Clinic visit": "53", "s1_LabID_CRCid": "73571", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "0.75", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "7646", "SubjectID": "27946", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.9", "FirstClinicVisit": "38608", "Age at Clinic visit": "69", "s1_LabID_CRCid": "69869", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.14", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.14"})
data.append({"Kindred": "7646", "SubjectID": "29247", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.1", "FirstClinicVisit": "38917", "Age at Clinic visit": "90", "s1_LabID_CRCid": "73687", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "7646", "SubjectID": "36832", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.6", "FirstClinicVisit": "39048", "Age at Clinic visit": "71", "s1_LabID_CRCid": "75060", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.19", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "7646", "SubjectID": "30461", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "20.5", "FirstClinicVisit": "39056", "Age at Clinic visit": "48", "s1_LabID_CRCid": "75108", "Oncoarray_date": "PENDING", "Smoke": "Y", "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1", "RR SMOKE": "1.19", "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.19"})
data.append({"Kindred": "7646", "SubjectID": "30515", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.3", "FirstClinicVisit": "39203", "Age at Clinic visit": "62", "s1_LabID_CRCid": "77562", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "7646", "SubjectID": "23527", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "29.1", "FirstClinicVisit": "38062", "Age at Clinic visit": "61", "s1_LabID_CRCid": "62213", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.24", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.24"})
data.append({"Kindred": "7646", "SubjectID": "30464", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.7", "FirstClinicVisit": "39092", "Age at Clinic visit": "46", "s1_LabID_CRCid": "75554", "Oncoarray_date": "PENDING", "Smoke": "3", "Alcohol": "4", "NSAID": "2", "HRT": None, "Exercise": "0.75", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1.13", "RR ALCOHOL": "1.07", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.29567156"})
data.append({"Kindred": "7646", "SubjectID": "29150", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "28.8", "FirstClinicVisit": "38590", "Age at Clinic visit": "59", "s1_LabID_CRCid": "70063", "Oncoarray_date": "PENDING", "Smoke": "10", "Alcohol": "No", "NSAID": "4", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.24", "RR SMOKE": "1.13", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.317128"})
data.append({"Kindred": "7646", "SubjectID": "30460", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "31.5", "FirstClinicVisit": "39175", "Age at Clinic visit": "50", "s1_LabID_CRCid": "76810", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "7646", "SubjectID": "30497", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "34.7", "FirstClinicVisit": "38888", "Age at Clinic visit": "47", "s1_LabID_CRCid": "73273", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.41"})
data.append({"Kindred": "7646", "SubjectID": "28476", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "33.7", "FirstClinicVisit": "38538", "Age at Clinic visit": "55", "s1_LabID_CRCid": "68809", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "7647", "SubjectID": "38360", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39336", "Age at Clinic visit": "44", "s1_LabID_CRCid": "79444", "Oncoarray_date": None, "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "35488", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "38994", "Age at Clinic visit": "71", "s1_LabID_CRCid": "74462", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "19835", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "3", "DeathYear": "2013", "BMI": None, "FirstClinicVisit": "37882", "Age at Clinic visit": "85", "s1_LabID_CRCid": "33036", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "36813", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39057", "Age at Clinic visit": "68", "s1_LabID_CRCid": "75125", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "36814", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39119", "Age at Clinic visit": "61", "s1_LabID_CRCid": "75887", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "38358", "Sex": "m", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39507", "Age at Clinic visit": "50", "s1_LabID_CRCid": "82071", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": None, "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "0"})
data.append({"Kindred": "7647", "SubjectID": "36794", "Sex": "M", "VitalStatus": "Deceased", "DeathMonth": "5", "DeathYear": "2009", "BMI": None, "FirstClinicVisit": "39087", "Age at Clinic visit": "75", "s1_LabID_CRCid": "75516", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "10", "HRT": None, "Exercise": "5", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "0.76", "Product of RR": "0.646"})
data.append({"Kindred": "7647", "SubjectID": "36791", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.3", "FirstClinicVisit": "39002", "Age at Clinic visit": "81", "s1_LabID_CRCid": "74606", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "19", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.77"})
data.append({"Kindred": "7647", "SubjectID": "37970", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "19.5", "FirstClinicVisit": "39209", "Age at Clinic visit": "71", "s1_LabID_CRCid": "78487", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "6", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.8"})
data.append({"Kindred": "7647", "SubjectID": "37232", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39170", "Age at Clinic visit": "58", "s1_LabID_CRCid": "77451", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "6", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.85"})
data.append({"Kindred": "7647", "SubjectID": "36801", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "22.7", "FirstClinicVisit": "39092", "Age at Clinic visit": "68", "s1_LabID_CRCid": "75621", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "2", "NSAID": "No", "HRT": "15", "Exercise": "No", "NOTE": None, "RR BMI": "1", "RR SMOKE": "1", "RR ALCOHOL": "1.07", "RR NSAID": "1", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.856"})
data.append({"Kindred": "7647", "SubjectID": "36783", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "8", "DeathYear": "2013", "BMI": "31.3", "FirstClinicVisit": "39050", "Age at Clinic visit": "75", "s1_LabID_CRCid": "75061", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "45", "HRT": "30", "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.77", "RR HRT": "0.8", "RR EXERCISE": "1", "Product of RR": "0.86856"})
data.append({"Kindred": "7647", "SubjectID": "38357", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39265", "Age at Clinic visit": "52", "s1_LabID_CRCid": "79445", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "2", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.94"})
data.append({"Kindred": "7647", "SubjectID": "37969", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39283", "Age at Clinic visit": "77", "s1_LabID_CRCid": "78748", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "3", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.94"})
data.append({"Kindred": "7647", "SubjectID": "36790", "Sex": "F", "VitalStatus": "Deceased", "DeathMonth": "12", "DeathYear": "2010", "BMI": "23.9", "FirstClinicVisit": "39029", "Age at Clinic visit": "85", "s1_LabID_CRCid": "75965", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "10", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.969"})
data.append({"Kindred": "7647", "SubjectID": "36812", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.4", "FirstClinicVisit": "39265", "Age at Clinic visit": "66", "s1_LabID_CRCid": "78490", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "7", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.85", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "0.969"})
data.append({"Kindred": "7647", "SubjectID": "37964", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39121", "Age at Clinic visit": "66", "s1_LabID_CRCid": "76928", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "7647", "SubjectID": "37468", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": None, "FirstClinicVisit": "39121", "Age at Clinic visit": "88", "s1_LabID_CRCid": "75964", "Oncoarray_date": None, "Smoke": "No", "Alcohol": "No ", "NSAID": "No", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": None, "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1"})
data.append({"Kindred": "7647", "SubjectID": "36781", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.7", "FirstClinicVisit": "39023", "Age at Clinic visit": "64", "s1_LabID_CRCid": "76524", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "2", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.1186"})
data.append({"Kindred": "7647", "SubjectID": "38361", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.4", "FirstClinicVisit": "39336", "Age at Clinic visit": "41", "s1_LabID_CRCid": "79447", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "1", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "7647", "SubjectID": "38187", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.4", "FirstClinicVisit": "39308", "Age at Clinic visit": "54", "s1_LabID_CRCid": "79448", "Oncoarray_date": "WGS", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "7647", "SubjectID": "37283", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "24.2", "FirstClinicVisit": "39107", "Age at Clinic visit": "79", "s1_LabID_CRCid": "76929", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "7647", "SubjectID": "38356", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "23.2", "FirstClinicVisit": "39281", "Age at Clinic visit": "53", "s1_LabID_CRCid": "78780", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No ", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.14", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.14"})
data.append({"Kindred": "7647", "SubjectID": "36795", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "25.8", "FirstClinicVisit": "39072", "Age at Clinic visit": "72", "s1_LabID_CRCid": "75749", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "7647", "SubjectID": "37966", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "26.5", "FirstClinicVisit": "39122", "Age at Clinic visit": "76", "s1_LabID_CRCid": "78488", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "No", "HRT": "No", "Exercise": "No", "NOTE": None, "RR BMI": "1.19", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "1", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.19"})
data.append({"Kindred": "7647", "SubjectID": "37233", "Sex": "M", "VitalStatus": "alive", "DeathMonth": None, "DeathYear": None, "BMI": "40.1", "FirstClinicVisit": "39104", "Age at Clinic visit": "55", "s1_LabID_CRCid": "76345", "Oncoarray_date": "PENDING", "Smoke": "No", "Alcohol": "No", "NSAID": "5", "HRT": None, "Exercise": "No", "NOTE": None, "RR BMI": "1.41", "RR SMOKE": "1", "RR ALCOHOL": "1", "RR NSAID": "0.94", "RR HRT": "1", "RR EXERCISE": "1", "Product of RR": "1.3254"})
data.append({"Kindred": "7647", "SubjectID": "36809", "Sex": "F", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "37.4", "FirstClinicVisit": "39161", "Age at Clinic visit": "54", "s1_LabID_CRCid": "76522", "Oncoarray_date": "PENDING", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": None, "RR EXERCISE": None, "Product of RR": "1.41"})
data.append({"Kindred": "7647", "SubjectID": "38359", "Sex": "M", "VitalStatus": "Alive", "DeathMonth": None, "DeathYear": None, "BMI": "30", "FirstClinicVisit": "39336", "Age at Clinic visit": "48", "s1_LabID_CRCid": "79446", "Oncoarray_date": "WGS", "Smoke": None, "Alcohol": None, "NSAID": None, "HRT": None, "Exercise": None, "NOTE": None, "RR BMI": "1.41", "RR SMOKE": None, "RR ALCOHOL": None, "RR NSAID": None, "RR HRT": "MALE", "RR EXERCISE": None, "Product of RR": "1.41"})



print(len(data))

350


In [54]:
data_polyp = []
data_polyp.append({"Kindred": "4562", "SubjectID": "10001885", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "49", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001885", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "49", "polypType": "Adenocarcinoma", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001885", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "49", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001885", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "32086", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "51", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001885", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "33948", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "56", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "59", "polypType": "Unknown", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "59", "polypType": "Unknown", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "59", "polypType": "Unknown", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "59", "polypType": "Unknown", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "59", "polypType": "Unknown", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31245", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "59", "polypType": "Unknown", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31306", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "59", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31306", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "59", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "31639", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "2", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "32013", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "61", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "32013", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "61", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001886", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36199", "ProcedureDate": "32721", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "63", "polypType": "Unknown", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001889", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35892", "ProcedureDate": "31266", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "54", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001889", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35892", "ProcedureDate": "31952", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "56", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001889", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35892", "ProcedureDate": "31952", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "56", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001889", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35892", "ProcedureDate": "32323", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "57", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001893", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "32939", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "59", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001893", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "34939", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Descending", "polypAge": "65", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10001893", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "36131", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "68", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002134", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "36131", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "73", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002134", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "36131", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "73", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002134", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36130", "ProcedureDate": "36131", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "73", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002136", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36271", "ProcedureDate": "36292", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002136", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36271", "ProcedureDate": "36292", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33428", "ProcType": "Flexible Sigmoidoscopy", "site": "Anus", "polypAge": "73", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": ">39mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33428", "ProcType": "Flexible Sigmoidoscopy", "site": "Anus", "polypAge": "73", "polypType": "Villous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33590", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "73", "polypType": "Lymphoid Hyperplasia", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33590", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "73", "polypType": "Lymphoid Hyperplasia", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33590", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "73", "polypType": "Lymphoid Hyperplasia", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002159", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36144", "ProcedureDate": "33590", "ProcType": "Colonoscopy", "site": "Ileum", "polypAge": "73", "polypType": "Lymphoid Hyperplasia", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002162", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36173", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "42", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002164", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36173", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "39", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36186", "ProcedureDate": "35873", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "45", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36186", "ProcedureDate": "36187", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "46", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36186", "ProcedureDate": "36187", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002186", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36187", "ProcedureDate": "36187", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "39", "polypType": "Lymphoid Follicle", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002186", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36187", "ProcedureDate": "36187", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "39", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002187", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36179", "ProcedureDate": "36130", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "42", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002187", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36179", "ProcedureDate": "36874", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "44", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002195", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002195", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002195", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002199", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36214", "ProcedureDate": "36215", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "46", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002199", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36214", "ProcedureDate": "36215", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002201", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36243", "ProcedureDate": "36243", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "51", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002202", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36242", "ProcedureDate": "36243", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "52", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002205", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "36215", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "43", "polypType": "Lymphoid Hyperplasia", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002224", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "34858", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002224", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "34858", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002224", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "35916", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "48", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002224", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "35916", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "48", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002225", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "36222", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002226", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "36222", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002226", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "37998", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "48", "polypType": "Hyperplastic", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002226", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36222", "ProcedureDate": "39307", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "51", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36264", "ProcedureDate": "36264", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "42", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35698", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35698", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "57", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35698", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Splenic Flexure", "polypAge": "57", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35709", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "57", "polypType": "Adenomatous", "polypNumber": "several", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35709", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35709", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35709", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002238", "OnDebsListOf296": "N", "FirstClinicVisit": "36234", "ProcedureDate": "35709", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "57", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002240", "OnDebsListOf296": "N", "FirstClinicVisit": "36207", "ProcedureDate": "36208", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002240", "OnDebsListOf296": "N", "FirstClinicVisit": "36207", "ProcedureDate": "36208", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002240", "OnDebsListOf296": "N", "FirstClinicVisit": "36207", "ProcedureDate": "36208", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002240", "OnDebsListOf296": "N", "FirstClinicVisit": "36207", "ProcedureDate": "36208", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002252", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36243", "ProcedureDate": "36243", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "74", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002252", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36243", "ProcedureDate": "36243", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "74", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002252", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36243", "ProcedureDate": "36243", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "74", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002253", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36229", "ProcedureDate": "35867", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002253", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36229", "ProcedureDate": "36238", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002254", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36230", "ProcedureDate": "36229", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002257", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36230", "ProcedureDate": "36229", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "58", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002263", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002264", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36250", "ProcedureDate": "36250", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002264", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36250", "ProcedureDate": "36250", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "55", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002265", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36292", "ProcedureDate": "36319", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "50", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002265", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36292", "ProcedureDate": "36319", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "50", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002278", "OnDebsListOf296": "N", "FirstClinicVisit": "36264", "ProcedureDate": "36264", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "36", "polypType": "No tissue identified at pathology", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4562", "SubjectID": "10002300", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36291", "ProcedureDate": "36292", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "40", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001871", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "65", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001871", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "65", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001871", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "36706", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "67", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001911", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "74", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001912", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001913", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35920", "ProcedureDate": "35921", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "44", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "34802", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "68", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35173", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "69", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "71", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "71", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "35942", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "71", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001916", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35941", "ProcedureDate": "36641", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "73", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "62", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "62", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "62", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "62", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001917", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "62", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35437", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "63", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35963", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35963", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35963", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "64", "polypType": "Hyperplastic", "polypNumber": "5", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "66", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001920", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "66", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001921", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "49", "polypType": "No tissue identified at pathology", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001921", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001925", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "47", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001925", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "47", "polypType": "Normal Colonic Mucosa", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001925", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35948", "ProcedureDate": "35949", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "47", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001926", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "80", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001926", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "80", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001926", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "80", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001926", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "80", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001934", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "34234", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "64", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001934", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "34234", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "64", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001934", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "34583", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "65", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001934", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "35998", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "68", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001934", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "35998", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "68", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001936", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35955", "ProcedureDate": "35956", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001938", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35508", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "65", "polypType": "Adenomatous", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001938", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35962", "ProcedureDate": "35508", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "65", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001939", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36120", "ProcedureDate": "34354", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "59", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001942", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36089", "ProcedureDate": "34437", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "41", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001942", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36089", "ProcedureDate": "35766", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "45", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001950", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36111", "ProcedureDate": "34926", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "69", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001955", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35969", "ProcedureDate": "35970", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "44", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001955", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35969", "ProcedureDate": "35970", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "44", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001956", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35969", "ProcedureDate": "35970", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001957", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "35998", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "47", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001960", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "35998", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "42", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001960", "OnDebsListOf296": "Yes", "FirstClinicVisit": "35997", "ProcedureDate": "35998", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "42", "polypType": "Normal Colonic Mucosa", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001962", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36011", "ProcedureDate": "36012", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "57", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001962", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36011", "ProcedureDate": "36012", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "57", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001965", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36060", "ProcedureDate": "36061", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "38", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001965", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36060", "ProcedureDate": "36061", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "38", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001966", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001966", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "60", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001966", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001967", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36053", "ProcedureDate": "35041", "ProcType": "Colonoscopy", "site": "Ileum", "polypAge": "46", "polypType": "Lymphoid Follicle", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "53", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "53", "polypType": "Normal Colonic Mucosa", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001968", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001970", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "58", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001970", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "58", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001970", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "58", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001970", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36032", "ProcedureDate": "36033", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "58", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001971", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36046", "ProcedureDate": "36047", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "43", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001977", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "39", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001977", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36025", "ProcedureDate": "36026", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "39", "polypType": "Normal Colonic Mucosa", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001999", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36011", "ProcedureDate": "36012", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "35", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001999", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36011", "ProcedureDate": "36012", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "35", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10001999", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36011", "ProcedureDate": "36012", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "35", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002051", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36111", "ProcedureDate": "35522", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "48", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002051", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36111", "ProcedureDate": "35522", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "48", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002051", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36111", "ProcedureDate": "35522", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "48", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002058", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36081", "ProcedureDate": "36082", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002058", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36081", "ProcedureDate": "36082", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002060", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36081", "ProcedureDate": "36082", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "47", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002070", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36088", "ProcedureDate": "36089", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "43", "polypType": "Inflammatory", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002072", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36088", "ProcedureDate": "36089", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "37", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002072", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36088", "ProcedureDate": "36089", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002073", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36088", "ProcedureDate": "36089", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "35", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002074", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36088", "ProcedureDate": "36089", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "39", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002093", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36109", "ProcedureDate": "36110", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "30", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002093", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36109", "ProcedureDate": "36110", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "30", "polypType": "Normal Colonic Mucosa", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002093", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36109", "ProcedureDate": "36110", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "30", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002093", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36109", "ProcedureDate": "36110", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "30", "polypType": "Other [benign]", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002096", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "49", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002096", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "49", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002096", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002096", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36482", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "49", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "33254", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "31", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "34416", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "34", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "34416", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "34", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "34960", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "35", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36117", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "38", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002099", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36117", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "38", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002100", "OnDebsListOf296": "N", "FirstClinicVisit": "36116", "ProcedureDate": "36117", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "52", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002155", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "40513", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "42", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002155", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "40513", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "42", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002155", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "40513", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "42", "polypType": "Unknown/No biopsy taken", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002156", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36173", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "32", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002156", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36173", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "32", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002156", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36992", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "34", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4586", "SubjectID": "10002156", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36172", "ProcedureDate": "36992", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "34", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10236", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39573", "ProcedureDate": "39550", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "68", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10577", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39371", "ProcedureDate": "39302", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "62", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10579", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39371", "ProcedureDate": "39372", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "38", "polypType": "Hyperplastic", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13750", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37369", "ProcedureDate": "37048", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13751", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37369", "ProcedureDate": "37370", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13753", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "46", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13753", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13753", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "46", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13753", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "13754", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "42255", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39518", "ProcedureDate": "39519", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "33", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36060", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "66", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36060", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "66", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": ">39mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36069", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "66", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36069", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "66", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "37566", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "70", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001891", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "38666", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "73", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10001894", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "63", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002085", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36102", "ProcedureDate": "36103", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002086", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "60", "polypType": "Lymphoid Follicle", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002086", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002086", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002086", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36137", "ProcedureDate": "36138", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002088", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36509", "ProcedureDate": "36509", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002088", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36509", "ProcedureDate": "36509", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002207", "OnDebsListOf296": "N", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "40", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002207", "OnDebsListOf296": "N", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "40", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002207", "OnDebsListOf296": "N", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "40", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002207", "OnDebsListOf296": "N", "FirstClinicVisit": "36179", "ProcedureDate": "36180", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "40", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "45", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "45", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "45", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36285", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002290", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36411", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "36306", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "36306", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "36306", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "36306", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "36306", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "55", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "39675", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002297", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36306", "ProcedureDate": "39675", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "64", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002516", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36628", "ProcedureDate": "36628", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "44", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002522", "OnDebsListOf296": "N", "FirstClinicVisit": "36628", "ProcedureDate": "36628", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "52", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002566", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36748", "ProcedureDate": "36747", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "58", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "4853", "SubjectID": "10002566", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36748", "ProcedureDate": "36747", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "58", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10834", "OnDebsListOf296": "N", "FirstClinicVisit": "36924", "ProcedureDate": "36922", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30629", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30629", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "59", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30697", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30697", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30697", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "60", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30697", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "30697", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "31012", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "31012", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "31434", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "31434", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "31434", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "62", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "34921", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "34921", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "34921", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "35009", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "35206", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "35373", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "72", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "35373", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "77", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "77", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "77", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "77", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "77", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10943", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "77", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11074", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36963", "ProcedureDate": "36964", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "57", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11074", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36963", "ProcedureDate": "36964", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11074", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36963", "ProcedureDate": "36964", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "57", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11180", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37040", "ProcedureDate": "37041", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11183", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37012", "ProcedureDate": "37013", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "47", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37081", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37081", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11185", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37081", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11513", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37019", "ProcedureDate": "37020", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "48", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11723", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11723", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11723", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "38580", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "11723", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "38580", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13986", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37152", "ProcedureDate": "37153", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "41", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13987", "OnDebsListOf296": "N", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13987", "OnDebsListOf296": "N", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13987", "OnDebsListOf296": "N", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13987", "OnDebsListOf296": "N", "FirstClinicVisit": "37161", "ProcedureDate": "37160", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "43", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13988", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "44", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "13988", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "44", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14073", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37187", "ProcedureDate": "37188", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "36", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14290", "OnDebsListOf296": "N", "FirstClinicVisit": "37187", "ProcedureDate": "37188", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "39", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14291", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "36", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14291", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "36", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14291", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "36", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14318", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14318", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14318", "OnDebsListOf296": "N", "FirstClinicVisit": "37173", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "41", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14552", "OnDebsListOf296": "N", "FirstClinicVisit": "37264", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "42", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14552", "OnDebsListOf296": "N", "FirstClinicVisit": "37264", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "42", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "14837", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37287", "ProcedureDate": "36103", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "73", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "32612", "OnDebsListOf296": "N", "FirstClinicVisit": "38980", "ProcedureDate": "38980", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "68", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "32612", "OnDebsListOf296": "N", "FirstClinicVisit": "38980", "ProcedureDate": "38980", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "68", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "32612", "OnDebsListOf296": "N", "FirstClinicVisit": "38980", "ProcedureDate": "38980", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "68", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32022", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "59", "polypType": "Villoglandular Adenoma", "polypNumber": "1", "polypSize": ">39mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32203", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "59", "polypType": "No tissue identified at pathology", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32203", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "59", "polypType": "AdenoCA [mucinous]", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32203", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "59", "polypType": "No tissue identified at pathology", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32492", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "60", "polypType": "Villoglandular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32492", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "60", "polypType": "Villoglandular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32492", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "60", "polypType": "Villoglandular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32496", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "60", "polypType": "Adenomatous", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32876", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "61", "polypType": "Unknown", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "32876", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "61", "polypType": "Unknown", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "35928", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "70", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "35928", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "70", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "72", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001897", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36285", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001961", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36250", "ProcedureDate": "36250", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001961", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36250", "ProcedureDate": "36250", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10001961", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36250", "ProcedureDate": "36250", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002322", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "36467", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002322", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "36467", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002322", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "36467", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002362", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36509", "ProcedureDate": "36299", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "50", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002376", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "35062", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002376", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "35062", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002376", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "37698", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002376", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "37698", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002376", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "37698", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "62", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002377", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36446", "ProcedureDate": "36446", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "68", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002378", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36460", "ProcedureDate": "32951", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "61", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002381", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "36467", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "67", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002381", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36467", "ProcedureDate": "37645", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "71", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002388", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36461", "ProcedureDate": "36460", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "47", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002388", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36461", "ProcedureDate": "36460", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "47", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002389", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36460", "ProcedureDate": "36467", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "45", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002540", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36957", "ProcedureDate": "36957", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "75", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002540", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36957", "ProcedureDate": "36957", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "75", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002548", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36643", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002548", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36643", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002548", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36643", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002548", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36643", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "53", "polypType": "No tissue identified at pathology", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002548", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36643", "ProcedureDate": "36663", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "53", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "85", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "85", "polypType": "Tubular Adenoma", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "85", "polypType": "Tubular Adenoma", "polypNumber": "5", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "85", "polypType": "Unknown/No biopsy taken", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "85", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "85", "polypType": "Unknown/No biopsy taken", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002552", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37119", "ProcedureDate": "37886", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "85", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002553", "OnDebsListOf296": "N", "FirstClinicVisit": "36740", "ProcedureDate": "35188", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002553", "OnDebsListOf296": "N", "FirstClinicVisit": "36740", "ProcedureDate": "35188", "ProcType": "Flexible Sigmoidoscopy", "site": "Rectum", "polypAge": "59", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002559", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36845", "ProcedureDate": "36845", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002559", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36845", "ProcedureDate": "36845", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "69", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002559", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36845", "ProcedureDate": "36845", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "32965", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "57", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "32965", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "57", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "32965", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "57", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "33827", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "59", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "67", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "67", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "67", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002578", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "67", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002579", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "34932", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002579", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "35017", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002579", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002587", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37110", "ProcedureDate": "37111", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002587", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37110", "ProcedureDate": "37111", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002608", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36845", "ProcedureDate": "36845", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "33826", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "66", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "33826", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "66", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "33826", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "66", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "33826", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "66", "polypType": "Adenomatous", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "33826", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "66", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "34213", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "67", "polypType": "Adenomatous", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "37167", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "75", "polypType": "Tubular Adenoma", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002621", "OnDebsListOf296": "N", "FirstClinicVisit": "37166", "ProcedureDate": "37167", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "75", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002622", "OnDebsListOf296": "N", "FirstClinicVisit": "36923", "ProcedureDate": "35870", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "46", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002622", "OnDebsListOf296": "N", "FirstClinicVisit": "36923", "ProcedureDate": "35870", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002622", "OnDebsListOf296": "N", "FirstClinicVisit": "36923", "ProcedureDate": "36922", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "49", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002623", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37287", "ProcedureDate": "35870", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "69", "polypType": "Adenomatous", "polypNumber": "2", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002877", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002877", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002898", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "44", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10002898", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36860", "ProcedureDate": "36859", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "44", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003000", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003002", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37061", "ProcedureDate": "37062", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "36", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36950", "ProcedureDate": "36950", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "39", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36950", "ProcedureDate": "36950", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "39", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36950", "ProcedureDate": "36950", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "39", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5234", "SubjectID": "10003006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36950", "ProcedureDate": "36950", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "39", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "7956", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38979", "ProcedureDate": "38980", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "28", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "35907", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "35907", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "72", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "35907", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "35907", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "75", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10068", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39112", "ProcedureDate": "37174", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "75", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "37664", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "52", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "37664", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "52", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "37664", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "52", "polypType": "Unknown/No biopsy taken", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "37664", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "52", "polypType": "Unknown/No biopsy taken", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "38847", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "38847", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "17660", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37664", "ProcedureDate": "38847", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "37734", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "37734", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "39400", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18006", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "39400", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18114", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "37740", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18114", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37733", "ProcedureDate": "37740", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18187", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37754", "ProcedureDate": "37755", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "32", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18188", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37754", "ProcedureDate": "37754", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "25", "polypType": "Lymphoid Follicle", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18349", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38931", "ProcedureDate": "37811", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "18349", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38931", "ProcedureDate": "38931", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "63", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21317", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38876", "ProcedureDate": "38467", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21317", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38876", "ProcedureDate": "38467", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21318", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39105", "ProcedureDate": "39106", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21318", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39105", "ProcedureDate": "39106", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21318", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39105", "ProcedureDate": "39106", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21318", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39105", "ProcedureDate": "39106", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "21318", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39105", "ProcedureDate": "39106", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "35706", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38876", "ProcedureDate": "38467", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "59", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "35735", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38888", "ProcedureDate": "38889", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "30", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "35737", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39070", "ProcedureDate": "39071", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "34", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "36732", "OnDebsListOf296": "N", "FirstClinicVisit": "43276", "ProcedureDate": "39099", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "40", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "37236", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39232", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "27", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "37236", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39232", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "27", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "40903", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39414", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "40903", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39414", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "40903", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39414", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "6", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002259", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36277", "ProcedureDate": "34523", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "38", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002259", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36277", "ProcedureDate": "34523", "ProcType": "Flexible Sigmoidoscopy", "site": "Rectosigmoid Junction", "polypAge": "38", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002259", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36277", "ProcedureDate": "35307", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "40", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002259", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36277", "ProcedureDate": "36047", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "42", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002259", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36277", "ProcedureDate": "36448", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "43", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002285", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "34530", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "41", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002285", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "34530", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "41", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002285", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36145", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "45", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002285", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36145", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "45", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002285", "OnDebsListOf296": "Yes", "FirstClinicVisit": "36284", "ProcedureDate": "36145", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "45", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "5275", "SubjectID": "10002533", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37685", "ProcedureDate": "38267", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "80", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "19834", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37880", "ProcedureDate": "33519", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "72", "polypType": "Adenocarcinoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "22836", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38874", "ProcedureDate": "38910", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "22839", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38464", "ProcedureDate": "37371", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "37", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "22839", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38464", "ProcedureDate": "37371", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "37", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29150", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38590", "ProcedureDate": "39063", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "60", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29151", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38609", "ProcedureDate": "35874", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29151", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38609", "ProcedureDate": "38609", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "64", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29152", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38590", "ProcedureDate": "36461", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "62", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29152", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38590", "ProcedureDate": "37698", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "66", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29152", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38590", "ProcedureDate": "39176", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "70", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29152", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38590", "ProcedureDate": "39176", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "70", "polypType": "Hyperplastic", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29246", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38602", "ProcedureDate": "38435", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "57", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "32380", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Adenocarcinoma In Situ", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "32380", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "72", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "32526", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "72", "polypType": "Adenocarcinoma In Situ", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "32539", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "72", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "33982", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "76", "polypType": "Adenomatous", "polypNumber": "several", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "33982", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "76", "polypType": "Adenomatous", "polypNumber": "several", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "33982", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "76", "polypType": "Hyperplastic", "polypNumber": "several", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "34316", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "77", "polypType": "Hyperplastic", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "35037", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "79", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "35037", "ProcType": "Colonoscopy", "site": "Colon, NOS", "polypAge": "79", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "35815", "ProcType": "Colonoscopy", "site": "Colon, Left", "polypAge": "81", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "29247", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38917", "ProcedureDate": "37847", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "87", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30459", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39175", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30459", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39175", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "69", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30459", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39175", "ProcedureDate": "37265", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30460", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39175", "ProcedureDate": "39176", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "50", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30464", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39092", "ProcedureDate": "39092", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "45", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30497", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38888", "ProcedureDate": "38889", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "47", "polypType": "Tubular Adenoma", "polypNumber": "several", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30497", "OnDebsListOf296": "Yes", "FirstClinicVisit": "38888", "ProcedureDate": "38889", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "47", "polypType": "Hyperplastic", "polypNumber": "several", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37173", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37173", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37173", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37173", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37173", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "several", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "30515", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39203", "ProcedureDate": "37566", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "56", "polypType": "Hyperplastic", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "35117", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "60", "polypType": "Inflammatory", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "35117", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "60", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "36236", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "36236", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "36236", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "64", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "36712", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "65", "polypType": "Tubular Adenoma", "polypNumber": "3", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7646", "SubjectID": "36832", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39048", "ProcedureDate": "38784", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "71", "polypType": "Inflammatory", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "19835", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37882", "ProcedureDate": "31625", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "68", "polypType": "Adenocarcinoma", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "19835", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37882", "ProcedureDate": "33526", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "73", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "19835", "OnDebsListOf296": "Yes", "FirstClinicVisit": "37882", "ProcedureDate": "33526", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "73", "polypType": "Normal Colonic Mucosa", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36781", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39023", "ProcedureDate": "38702", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "62", "polypType": "Adenocarcinoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36783", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39050", "ProcedureDate": "38777", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "74", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "34353", "ProcType": "Colonoscopy", "site": "Colon, NOS", "polypAge": "72", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "35262", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "74", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "36339", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "77", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "37754", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "81", "polypType": "Adenocarcinoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "39276", "ProcType": "Colonoscopy", "site": "Anastomosis", "polypAge": "85", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36790", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39029", "ProcedureDate": "39276", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "85", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36791", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39002", "ProcedureDate": "35934", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "73", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "35166", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "63", "polypType": "Unknown/No biopsy taken", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "35173", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "63", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "37102", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "68", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "38714", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "72", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "38714", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "72", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36794", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39087", "ProcedureDate": "38714", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "72", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": ">39mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36795", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39072", "ProcedureDate": "37917", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36795", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39072", "ProcedureDate": "37917", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36795", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39072", "ProcedureDate": "37917", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "69", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36795", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39072", "ProcedureDate": "38244", "ProcType": "Flexible Sigmoidoscopy", "site": "Colon, Sigmoid", "polypAge": "69", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36795", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39072", "ProcedureDate": "38244", "ProcType": "Flexible Sigmoidoscopy", "site": "Rectosigmoid Junction", "polypAge": "69", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36809", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39161", "ProcedureDate": "39162", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36809", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39161", "ProcedureDate": "39162", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36809", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39161", "ProcedureDate": "39162", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36809", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39161", "ProcedureDate": "39162", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "53", "polypType": "Other [benign]", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36812", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "34282", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "52", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36812", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "35492", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "55", "polypType": "Villous Adenoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36812", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "36340", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "57", "polypType": "Lymphoid Aggregate Formation", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36812", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "37777", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "61", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36812", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "37777", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "61", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36814", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39119", "ProcedureDate": "35431", "ProcType": "Colonoscopy", "site": "Colon, NOS", "polypAge": "50", "polypType": "Adenomatous", "polypNumber": "unknown", "polypSize": "Unknown"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36814", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39119", "ProcedureDate": "36425", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "53", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "36814", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39119", "ProcedureDate": "37585", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "56", "polypType": "Other [benign]", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "37172", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "52", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "37172", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "52", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "37172", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "52", "polypType": "Tubulovillous Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "38366", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "55", "polypType": "Tubulovillous Adenoma", "polypNumber": "4", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "38366", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "55", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "38366", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "55", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "38740", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37232", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39170", "ProcedureDate": "38740", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "56", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37233", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39148", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "55", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38145", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "63", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38145", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "63", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38145", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "63", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38145", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "63", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "30-39mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38588", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "64", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38588", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "64", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "38588", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "64", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Colon, Descending", "polypAge": "65", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Colon, Right", "polypAge": "65", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "65", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Rectosigmoid Junction", "polypAge": "65", "polypType": "Adenomatous", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "65", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37964", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39121", "ProcedureDate": "39085", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "65", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37966", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39122", "ProcedureDate": "38707", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "74", "polypType": "Lymphoid Aggregate Formation", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37966", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39122", "ProcedureDate": "38707", "ProcType": "Colonoscopy", "site": "Colon, Hepatic Flexure", "polypAge": "74", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37969", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39283", "ProcedureDate": "37390", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "72", "polypType": "Adenocarcinoma In Situ", "polypNumber": "1", "polypSize": "20-29mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37969", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39283", "ProcedureDate": "37390", "ProcType": "Colonoscopy", "site": "Colon, Cecum", "polypAge": "72", "polypType": "Unknown/Lost Polyp", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37969", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39283", "ProcedureDate": "37390", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "72", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "37969", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39283", "ProcedureDate": "38209", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "74", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38357", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "37277", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "46", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38357", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "37277", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "46", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38357", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "37277", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "46", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38357", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39265", "ProcedureDate": "38769", "ProcType": "Colonoscopy", "site": "Colon, Splenic Flexure", "polypAge": "50", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "50", "polypType": "Tubulovillous Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38358", "OnDebsListOf296": "Yes", "FirstClinicVisit": "43276", "ProcedureDate": "39192", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "50", "polypType": "Hyperplastic", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38359", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "48", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "<5mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38359", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Colon, Sigmoid", "polypAge": "48", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38359", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Colon, Transverse", "polypAge": "48", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38359", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "48", "polypType": "Hyperplastic", "polypNumber": "2", "polypSize": "5-9mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38359", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Rectum", "polypAge": "48", "polypType": "Tubulovillous Adenoma", "polypNumber": "2", "polypSize": "10-19mm"})
data_polyp.append({"Kindred": "7647", "SubjectID": "38361", "OnDebsListOf296": "Yes", "FirstClinicVisit": "39336", "ProcedureDate": "39337", "ProcType": "Colonoscopy", "site": "Colon, Ascending", "polypAge": "40", "polypType": "Tubular Adenoma", "polypNumber": "1", "polypSize": "5-9mm"})
print(len(data_polyp))

570


In [58]:
def isfloat(x):
    try:
        a = float(x)
    except (TypeError, ValueError):
        return False
    else:
        return True

def isint(x):
    try:
        a = float(x)
        b = int(a)
    except (TypeError, ValueError):
        return False
    else:
        return a == b
    
data2 = []
for d in data:
    d2 = {}
    for k0, v in d.items():
        k = k0.lower()
        k = k.replace(' ', '_')
        if v != None:
            if isint(v):
                d2[k] = int(v)
            elif isfloat(v):
                d2[k] = float(v)
            else:
                d2[k] = v
        else:
            d2[k] = v
    data2.append(d2)
print(len(data2))

data_polyp2 = []
for d in data_polyp:
    d2 = {}
    for k0, v in d.items():
        k = k0.lower()
        k = k.replace(' ', '_')
        if v != None:
            if isint(v):
                d2[k] = int(v)
            elif isfloat(v):
                d2[k] = float(v)
            else:
                d2[k] = v
        else:
            d2[k] = v
    data_polyp2.append(d2)
    
print(len(data_polyp2))   

350
570


In [61]:
headers_nonpolyp = list(data2[0].keys())
headers_polyp = list(data_polyp2[0].keys())
print(headers_nonpolyp)
print(headers_polyp)


['kindred', 'subjectid', 'sex', 'vitalstatus', 'deathmonth', 'deathyear', 'bmi', 'firstclinicvisit', 'age_at_clinic_visit', 's1_labid_crcid', 'oncoarray_date', 'smoke', 'alcohol', 'nsaid', 'hrt', 'exercise', 'note', 'rr_bmi', 'rr_smoke', 'rr_alcohol', 'rr_nsaid', 'rr_hrt', 'rr_exercise', 'product_of_rr']
['kindred', 'subjectid', 'ondebslistof296', 'firstclinicvisit', 'proceduredate', 'proctype', 'site', 'polypage', 'polyptype', 'polypnumber', 'polypsize']


In [63]:
out_dir = os.path.join("output")
os.makedirs(out_dir, exist_ok=1)

out_p1 = os.path.join(out_dir, "non_polyp.json")
with open(out_p1, 'w+') as f:
    json.dump({"headers": headers_nonpolyp, "data": data2}, f, indent=4)
    print(out_p1)
    

out_p2 = os.path.join(out_dir, "polyp.json")
with open(out_p2, 'w+') as f:
    json.dump({"headers": headers_polyp, "data": data_polyp2}, f, indent=4)
    print(out_p2)


output\non_polyp.json
output\polyp.json
